In [1]:
import requests
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
from datetime import datetime


url="http://sarthak.nhmmp.gov.in/covid/facility-bed-occupancy-details/?show=200&pagenum=1&district_id=1&facility_org_type=0&facility=0"

r = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c=r.content
soup=BeautifulSoup(c,"html.parser")

name=soup.find_all("div",{"class":"hospitalname"})

In [2]:
def get_hosp_name(nm):
    start=0
    end=0
    for a in range(len(nm)):
        if nm[a]==">" and start==0:
            start=a+1
        if nm[a]=="/" and end==0:
            end=a
    return str(nm[start:end])

In [3]:
def read_label(nam):
    x=0
    y=0
    for a in range(len(nam)):
        if nam[a]==">" and x==0:
            x=a+1
        if nam[a]=="<" and y==0:
            y=a
    return str(nam[x:y])

In [4]:
def tot_beds(nam):
    x=0
    y=0
    for a in range(1,len(nam)):
        if nam[a]==">" and x==0:
            x=a+1
        if nam[a]=="<" and y==0:
            y=a

    return nam[x:y]

In [5]:
def decode(st):
    x=0
    y=0
    for a in range(len(st)):
        if st[a]=="/":
            x=a-1
            y=a+2
            break

    return[st[:x],st[y:]]

In [6]:
def decode_date(st):
    start=0
    end=0
    for i in range(len(st)):
        try:
            int(st[i])
            start=i
            break
        except:
            pass
    for i in range(len(st),0,-1):
        try:
            int(st[i])
            end=i+1
            break
        except:
            pass
        
    return st[start:end]

In [7]:
hospital_list=[]
for a in range(len(name)):
    hospital_list.append(get_hosp_name(str(name[a])))

In [8]:
beds=soup.find_all("td",{"class":"text-center"})

isolation=[]
oxygen=[]
icu=[]
for a in range(len(beds)):
    labels=beds[a].find_all("label")
    for b in range(3):
        labels[b]=str(labels[b])
    icu.append(int(read_label(labels[2])))
    oxygen.append(int(read_label(labels[1])))
    isolation.append(int(read_label(labels[0])))

In [9]:
total=soup.find_all("span",{"class":"badge bed-status"})
total_beds=[]
remaining_beds=[]
ret=[]

for a in range(len(total)):
    l=decode(tot_beds(str(total[a])))
    total_beds.append(int(l[0]))
    remaining_beds.append(int(l[1]))

In [10]:
total=soup.find_all("div",{"class":"last-updated"})
last_updated=[]
for a in range(len(total)):
    st=decode_date(str(total[a]))
    st=st.replace(",","")
    date_time_str = st
    print(st)
    date_time_obj = dt.datetime.strptime(date_time_str, '%d-%m-%Y %H:%M:%S')
    last_updated.append(date_time_obj)

25-05-2021 11:31:17
25-05-2021 10:28:30
25-05-2021 10:20:33
25-05-2021 15:05:28
25-05-2021 14:05:29
25-05-2021 10:17:05
25-05-2021 16:38:01
25-05-2021 09:34:19
25-05-2021 08:30:51
25-05-2021 12:34:02
25-05-2021 09:24:02
25-05-2021 08:50:09
25-05-2021 09:09:23
25-05-2021 10:37:16
25-05-2021 08:05:47
25-05-2021 12:57:54
25-05-2021 09:38:45
25-05-2021 09:15:08
25-05-2021 10:10:28
25-05-2021 09:30:28
25-05-2021 07:47:38
25-05-2021 10:27:44
25-05-2021 09:19:21
25-05-2021 08:32:30
25-05-2021 11:25:59
25-05-2021 12:12:14
25-05-2021 16:41:23
25-05-2021 16:36:18
25-05-2021 08:07:44
25-05-2021 13:08:50
25-05-2021 10:30:16
25-05-2021 16:26:13
25-05-2021 15:48:20
25-05-2021 09:55:27
25-05-2021 08:16:58
25-05-2021 11:03:52
25-05-2021 08:28:15
25-05-2021 08:06:45
25-05-2021 08:55:16
25-05-2021 13:01:40
25-05-2021 07:48:42
25-05-2021 11:14:31
25-05-2021 10:38:18
25-05-2021 09:30:13
25-05-2021 09:57:11
25-05-2021 10:58:04
25-05-2021 08:30:20
25-05-2021 08:27:49
25-05-2021 09:14:23
25-05-2021 14:18:29


In [11]:
df = pd.DataFrame([hospital_list,total_beds,remaining_beds,icu,oxygen,isolation,last_updated])
df=df.T
df.columns =['Hospital','Total','Remaining','ICU','Oxygen','Isolation','Last Updated']

In [12]:
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H%M")

d=df.sum(axis = 0, skipna = True)
d[0]="Total"
df=df.append(other=d,ignore_index=True)

In [13]:
df.to_csv("data/"+dt_string+".csv", index=False)

In [14]:
df[df["ICU"]>0]

,Hospital,Total,Remaining,ICU,Oxygen,Isolation,Last Updated
0,Carewell Multispeciality Hospital,55,22,6,16,0,2021-05-25 11:31:17
1,GMC Bhopal - DCH,680,510,183,215,112,2021-05-25 10:28:30
2,Chirayu Medical College,1100,400,60,210,130,2021-05-25 10:20:33
3,RKDF Medical College Hospital &amp; Research C...,225,121,11,110,0,2021-05-25 15:05:28
6,All India Institute of Medical Sciences,500,188,21,167,0,2021-05-25 16:38:01
...,...,...,...,...,...,...,...
129,Leelawati Memorial Hospital,30,18,2,4,12,2021-05-25 08:17:17
130,LBS BERASIA,18,15,6,4,5,2021-05-25 09:47:45
131,Haque Multispeciality and Trauma Center,20,16,7,9,0,2021-05-25 08:33:36
134,FUTURE MULTISPECIALITY HOSPITAL &amp; TRAUMA C...,17,15,8,5,2,2021-05-25 09:21:26


In [15]:
df[df["Oxygen"]>0]

,Hospital,Total,Remaining,ICU,Oxygen,Isolation,Last Updated
0,Carewell Multispeciality Hospital,55,22,6,16,0,2021-05-25 11:31:17
1,GMC Bhopal - DCH,680,510,183,215,112,2021-05-25 10:28:30
2,Chirayu Medical College,1100,400,60,210,130,2021-05-25 10:20:33
3,RKDF Medical College Hospital &amp; Research C...,225,121,11,110,0,2021-05-25 15:05:28
4,Narmada Trauma Centre,40,10,0,6,4,2021-05-25 14:05:29
...,...,...,...,...,...,...,...
131,Haque Multispeciality and Trauma Center,20,16,7,9,0,2021-05-25 08:33:36
132,AMAN HOSPITAL AND TRAUMA CENTRE,6,6,0,4,2,2021-05-25 14:44:35
133,Dirghayu Covid Care Center,30,30,0,30,0,2021-05-25 09:12:13
134,FUTURE MULTISPECIALITY HOSPITAL &amp; TRAUMA C...,17,15,8,5,2,2021-05-25 09:21:26


In [16]:
df[df["Isolation"]>0]

,Hospital,Total,Remaining,ICU,Oxygen,Isolation,Last Updated
1,GMC Bhopal - DCH,680,510,183,215,112,2021-05-25 10:28:30
2,Chirayu Medical College,1100,400,60,210,130,2021-05-25 10:20:33
4,Narmada Trauma Centre,40,10,0,6,4,2021-05-25 14:05:29
7,SILVER LINE,12,11,0,5,6,2021-05-25 09:34:19
9,RAJDEEP Hospital,20,6,4,1,1,2021-05-25 12:34:02
...,...,...,...,...,...,...,...
129,Leelawati Memorial Hospital,30,18,2,4,12,2021-05-25 08:17:17
130,LBS BERASIA,18,15,6,4,5,2021-05-25 09:47:45
132,AMAN HOSPITAL AND TRAUMA CENTRE,6,6,0,4,2,2021-05-25 14:44:35
134,FUTURE MULTISPECIALITY HOSPITAL &amp; TRAUMA C...,17,15,8,5,2,2021-05-25 09:21:26


In [17]:
a=df["Remaining"][(len(df)-1)]*100/df["Total"][(len(df)-1)]
a=round(a,2)
print("%age beds remaining:",a,"%")

%age beds remaining: 66.91 %


In [18]:
print

<function print>

In [19]:
df

,Hospital,Total,Remaining,ICU,Oxygen,Isolation,Last Updated
0,Carewell Multispeciality Hospital,55,22,6,16,0,2021-05-25 11:31:17
1,GMC Bhopal - DCH,680,510,183,215,112,2021-05-25 10:28:30
2,Chirayu Medical College,1100,400,60,210,130,2021-05-25 10:20:33
3,RKDF Medical College Hospital &amp; Research C...,225,121,11,110,0,2021-05-25 15:05:28
4,Narmada Trauma Centre,40,10,0,6,4,2021-05-25 14:05:29
...,...,...,...,...,...,...,...
131,Haque Multispeciality and Trauma Center,20,16,7,9,0,2021-05-25 08:33:36
132,AMAN HOSPITAL AND TRAUMA CENTRE,6,6,0,4,2,2021-05-25 14:44:35
133,Dirghayu Covid Care Center,30,30,0,30,0,2021-05-25 09:12:13
134,FUTURE MULTISPECIALITY HOSPITAL &amp; TRAUMA C...,17,15,8,5,2,2021-05-25 09:21:26
